## 라이브러리 설치

In [1]:
!pip install langchain sentence-transformers unstructured pypdf pdf2image docx2txt pdfminer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 

In [2]:
pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.9 MB/s eta 0:00:00


In [3]:
!pip install -q langchain openai langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.2 MB/s eta 0:00:00


이 부분은 사실 web을 외부데이터소스로 쓰는 경우에는 안해도 되는데, pdf파일이라던가 또는 txt등의 drive에 저장해서 쓰는 외부 파일은 마운트 해주는 것이 좋다고 생각해서 해당 코드를 추가했다.

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.environ["OPENAI_API_KEY"]='YOUR OPENAI API KEY'

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI

## Load the websites or urls and split(Load and Split)

In [6]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 57.7 MB/s eta 0:00:00


In [7]:
loader=WebBaseLoader([
    "https://en.wikipedia.org/wiki/Chocolate",
    "https://www.livescience.com/61754-chocolate-facts.html",
    "https://foodinsight.org/lets-learn-about-chocolate/",
    "https://www.medicalnewstoday.com/articles/270272#benefits"
    ])#초콜릿에 대한 문서
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)#recursive써야지 더 문장을 매끄럽게 chunk할 수 있어서 사용
docs=loader.load_and_split(text_splitter=text_splitter)
embeddings=OpenAIEmbeddings()#Embedding vectordb
vectorstore=FAISS.from_documents(docs,embeddings)
retriever=vectorstore.as_retriever()#vectorstore를 retriever로 사용

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
llm=ChatOpenAI(
    temperature=0.1,#temperature일관성 0일수록 일관성 있지만, 조금의 변화를 주기위해 0.1로 설정
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model_name="gpt-3.5-turbo",
    max_tokens=1000,
    streaming=True,#타이핑하듯이 보이게 하기
    verbose=True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## Multiquery retriever  
:사용자가 애매하거나 추상적인 질문을 해도 답할 수 있도록 질문을 여러 개로 생성해서 relevant document를 가져오도록 함

In [9]:
from langchain.retrievers.multi_query import MultiQueryRetriever#
question="what happens when you eat chocolate?"#추상적인 질문
retriever_from_llm=MultiQueryRetriever.from_llm(#어떤 질문들이 생성될 수 있는지를 보기
    retriever=retriever,llm=llm#vectordb를 검색기로 활용,llm=llm(위에서 정의한 거 사용)
)

In [10]:
unique_docs=retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)#question과 유사한 질문들을 retriever에서 찾아내기

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


9

In [11]:
unique_docs

[Document(metadata={'source': 'https://www.medicalnewstoday.com/articles/270272#benefits', 'title': 'Chocolate: Health benefits, facts, and research', 'description': 'Who might once have thought that chocolate could be so much more than a delicious but ultimately naughty treat? Well, research continues to show that eating chocolate, especially the dark variety high in cocoa, can have many benefits, including stopping bad cholesterol and cognitive decline. Read on to discover all.', 'language': 'en'}, page_content='some negative effects, too.Weight gain: Some studies suggest that chocolate consumption is linked to lower body mass index (BMI) and central body fat. However, chocolate can have a high calorie count due to its sugar and fat content. Anyone who is trying to slim down or maintain their weight should limit their chocolate consumption and check the label of their favorite product.Sugar content: The high sugar content of most chocolate can also be a cause of tooth decay.Migraine 

## Q&A chain을 사용해서 쿼리에 대한 답변 얻기

In [12]:
from langchain.chains.question_answering import load_qa_chain
chain=load_qa_chain(llm,chain_type="stuff",verbose=True)

In [13]:
from langchain.prompts import ChatPromptTemplate

필요에 따라 프롬프트 template을 쓸 수도 있다.

In [22]:
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import RetrievalQA

In [15]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",#stuff하는 이유는 청크타입 500으로 작게 설정해서 여러 청크를 참고한다고 하더라도, 1500토큰정도만 사용->max token을 넘지 않음
    #만약 청크가 길다면,map-reduce사용하는 것이 좋을듯?
    retriever=retriever,
    return_source_documents=True)#질문한 것에 대해서 답변할때, 어떤 문서의 소스인지 주석처리하게 함
#
query="What happens when you eat chocolate?"
result=qa(query)
print(result['result'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


When you eat chocolate, it can have both positive and negative effects on your health. Some studies suggest that chocolate consumption is linked to lower body mass index (BMI) and central body fat, but it can also contribute to weight gain due to its high calorie count from sugar and fat content. The high sugar content in chocolate can also lead to tooth decay. Additionally, chocolate has been associated with conditions like diabetes, coronary heart disease, and hypertension. However, chocolate is believed to contain high levels of antioxidants, and some studies have suggested it could lower cholesterol levels and prevent memory decline. So, the effects of eating chocolate can vary depending on the individual and the amount consumed.


In [16]:
result

{'query': 'What happens when you eat chocolate?',
 'result': 'When you eat chocolate, it can have both positive and negative effects on your health. Some studies suggest that chocolate consumption is linked to lower body mass index (BMI) and central body fat, but it can also contribute to weight gain due to its high calorie count from sugar and fat content. The high sugar content in chocolate can also lead to tooth decay. Additionally, chocolate has been associated with conditions like diabetes, coronary heart disease, and hypertension. However, chocolate is believed to contain high levels of antioxidants, and some studies have suggested it could lower cholesterol levels and prevent memory decline. So, the effects of eating chocolate can vary depending on the individual and the amount consumed.',
 'source_documents': [Document(metadata={'source': 'https://en.wikipedia.org/wiki/Chocolate', 'title': 'Chocolate - Wikipedia', 'language': 'en'}, page_content='Eating chocolate'),
  Document(